# Data Patch

In [1]:
import sys
sys.path.insert(0, '/home/jjian03/anaconda3/lib/python3.7/site-packages')

## Read records line by line

In [2]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
import gc


class Formatter:
    @staticmethod
    def get_timestamp(format="%Y/%m/%d %H:%M:%S"):
        return str((datetime.now().strftime(format)))

def load_json(buffer, path, buffer_size=5000):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            buffer.append(parse_json(json.loads(line)))
            if (len(buffer)+1) % buffer_size == 0:
                df = integrate_dataframe(buffer)
                buffer.clear()
                yield df
                gc.collect()

        if len(buffer) > 0:
            df = integrate_dataframe(buffer)
            buffer.clear()
            yield df

def safe_get_attr(json_obj, key):
    return json_obj[key] if key in json_obj else None

def parse_json(json_obj):
    _id = [v for k, v in safe_get_attr(json_obj, '_id').items()][0]
    df = pd.DataFrame({
        'id': _id

        , 'pmid': safe_get_attr(json_obj, 'pmid')
        , 'doi': safe_get_attr(json_obj, 'doi')
    }, index=[_id])

    # Type conversion
    for col_name in ['id'
                     , 'pmid'
                     , 'doi'
                    ]:
        df.loc[:, col_name] = df[col_name].astype(str)
    return df

def integrate_dataframe(buffer):
    df = pd.DataFrame()
    for row in buffer:
        df = df.append(row)
    return df


In [3]:
json_path = '/home/jjian03/data/website_quality_repository_pmid.json'

buffer = list()
for item in load_json(buffer, json_path):
    display(item.head())
    break

,id,pmid,doi
5ecd87d6150a1889d7f738cd,5ecd87d6150a1889d7f738cd,28232771,10.1177/0539018416675074
5ecd87d6150a1889d7f738ce,5ecd87d6150a1889d7f738ce,25183238,10.1016/j.vetmic.2014.07.026
5ecd87d6150a1889d7f738cf,5ecd87d6150a1889d7f738cf,25307940,10.1016/j.vetmic.2014.09.012
5ecd87d6150a1889d7f738d0,5ecd87d6150a1889d7f738d0,24080352,10.1016/j.vetmic.2013.07.034
5ecd87d6150a1889d7f738d1,5ecd87d6150a1889d7f738d1,25183238,10.1016/j.vetmic.2014.07.026


## Connect to Spark

In [4]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

try:
    print(spark.version)
except NameError as e:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.\
        config('spark.app.name', 'mongodb_migration_pmid').\
        config('spark.dynamicAllocation.enabled','true').\
        config('spark.dynamicAllocation.maxExecutors','50').\
        config('spark.dynamicAllocation.executorIdleTimeout','30s').\
        config('spark.driver.maxResultSize', '8g').\
        config('spark.driver.memory', '50g').\
        config('spark.executor.memory', '20g').\
        config('spark.task.maxFailures', '3').\
        config('spark.yarn.am.memory', '8g').\
        config("spark.rpc.message.maxSize", "1024"). \
        config('spark.yarn.max.executor.failures', '3').\
        config('spark.kryoserializer.buffer.max','1024m').\
        config('spark.yarn.executor.memoryOverhead', '50g').\
        getOrCreate()
    print(spark.version)
sc = spark.sparkContext
spark_sql = SQLContext(sc)


2.3.0.cloudera3


### Test Connection

In [18]:
def load_dataset(spark, path, name):
    spark.read.parquet(path).registerTempTable(name)    
    
mag = load_dataset(spark, '/user/lliang06/icon/MAG_publication_features.parquet', 'mag')
try:
    load_dataset(spark, '/user/jjian03/WebResourceQuality.parquet', 'WebResourceQuality')
except:
    pass

spark_sql.sql('''
    SELECT *
    FROM mag
''').limit(10).toPandas()

,PaperId,total_num_of_paper_citing,total_num_of_author_citing,total_num_of_affiliation_citing,total_num_of_journal_citing,total_num_of_author_self_citation,total_num_of_affiliation_self_citation,total_num_of_journal_self_citation,avg_year,min_year,...,median,num_of_author,num_of_author_citing,num_of_affiliation_citing,num_of_journal_citing,avg_hindex,first_author_hindex,last_author_hindex,avg_mid_author_hindex,paper_unique_affiliation
0,4364,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None
1,12793,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,1.000000,NaN,NaN,0.0,None
2,12793,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,1.000000,NaN,NaN,0.0,None
3,21527,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,4.000000,2.0,2.0,NaN,None
4,21527,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,4.000000,2.0,2.0,NaN,None
5,21757,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,3.800000,14.0,14.0,1.0,None
6,21757,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,3.800000,14.0,14.0,1.0,None
7,23267,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,1.666667,2.0,2.0,0.5,None
8,23267,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,1.666667,2.0,2.0,0.5,None
9,23758,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None


## Insert& Append to the Parquet File

In [6]:
from pyspark.sql.types import *
from pyspark.sql import Row


schema = StructType([
        StructField("id", StringType(), True),
        StructField("pmid", StringType(), True),
        StructField("doi", StringType(), True),

])


In [8]:
buffer = list()
cnt = 0
for item in load_json(buffer, json_path):
    if len(item) == 0:
        continue
#     item = item[['id', 'pmid', 'doi']]
    cnt+=len(item)
    print(f'{Formatter.get_timestamp()} - {cnt}')
    df_spark = spark_sql.createDataFrame(item, schema)
    try:
        df_spark.write.mode('append').parquet('/user/jjian03/WebResourceQuality_pmid.parquet')
    except Exception as e:
        print(f'Error: {str(e)}')
        for idx, row in item.iterrows():
            df_spark = spark_sql.createDataFrame(Row(row.tolist()), schema)
            df_spark.write.mode('append').parquet('/user/jjian03/WebResourceQuality_pmid.parquet')

2020/06/16 13:55:51 - 4999
2020/06/16 13:56:10 - 9998
2020/06/16 13:56:26 - 14997
2020/06/16 13:56:38 - 19996
2020/06/16 13:56:49 - 24995
2020/06/16 13:57:01 - 29994
2020/06/16 13:57:12 - 34993
2020/06/16 13:57:24 - 39992
2020/06/16 13:57:36 - 44991
2020/06/16 13:57:47 - 49990
2020/06/16 13:57:59 - 54989
2020/06/16 13:58:11 - 59988
2020/06/16 13:58:22 - 64987
2020/06/16 13:58:34 - 69986
2020/06/16 13:58:46 - 74985
2020/06/16 13:58:57 - 79984
2020/06/16 13:59:10 - 84983
2020/06/16 13:59:21 - 89982
2020/06/16 13:59:33 - 94981
2020/06/16 13:59:44 - 99980
2020/06/16 13:59:56 - 104979
2020/06/16 14:00:07 - 109978
2020/06/16 14:00:19 - 114977
2020/06/16 14:00:31 - 119976
2020/06/16 14:00:43 - 124975
2020/06/16 14:00:54 - 129974
2020/06/16 14:01:06 - 134973
2020/06/16 14:01:17 - 139972
2020/06/16 14:01:29 - 144971
2020/06/16 14:01:40 - 149970
2020/06/16 14:01:52 - 154969
2020/06/16 14:02:04 - 159968
2020/06/16 14:02:15 - 164967
2020/06/16 14:02:27 - 169966
2020/06/16 14:02:39 - 174965
2020/06

2020/06/16 14:50:37 - 1409718
2020/06/16 14:50:48 - 1414717
2020/06/16 14:51:00 - 1419716
2020/06/16 14:51:11 - 1424715
2020/06/16 14:51:23 - 1429714
2020/06/16 14:51:34 - 1434713
2020/06/16 14:51:46 - 1439712
2020/06/16 14:51:57 - 1444711
2020/06/16 14:52:09 - 1449710
2020/06/16 14:52:20 - 1454709
2020/06/16 14:52:32 - 1459708
2020/06/16 14:52:43 - 1464707
2020/06/16 14:52:55 - 1469706
2020/06/16 14:53:06 - 1474705
2020/06/16 14:53:18 - 1479704
2020/06/16 14:53:29 - 1484703
2020/06/16 14:53:41 - 1489702
2020/06/16 14:53:52 - 1494701
2020/06/16 14:54:04 - 1499700
2020/06/16 14:54:15 - 1504699
2020/06/16 14:54:27 - 1509698
2020/06/16 14:54:38 - 1514697
2020/06/16 14:54:50 - 1519696
2020/06/16 14:55:02 - 1524695
2020/06/16 14:55:14 - 1529694
2020/06/16 14:55:25 - 1534693
2020/06/16 14:55:37 - 1539692
2020/06/16 14:55:48 - 1544691
2020/06/16 14:56:00 - 1549690
2020/06/16 14:56:13 - 1554689
2020/06/16 14:56:24 - 1559688
2020/06/16 14:56:36 - 1564687
2020/06/16 14:56:47 - 1569686
2020/06/16